In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
import tensorflow as tf

# Set random seed for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

# Load the dataset
from google.colab import drive
drive.mount('/content/drive')
file_path = '/content/drive/MyDrive/withouticorvalcheack.xlsx'
data = pd.read_excel(file_path)

# Define columns based on types
binary_features = ['Gendermale1', 'BloodPressure']
numeric_features = ['Age', 'QualityofSleep', 'PhysicalActivityLevel', 'HeartRate']
categorical_features = ['BMICategory']

target = 'SleepDisorder'

# Define ColumnTransformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('binary', MinMaxScaler(), binary_features),
        ('numeric', StandardScaler(), numeric_features),
        ('categorical', OneHotEncoder(), categorical_features),
    ]
)

# Separate features (X) and target variable (y)
X = data.drop(columns=[target])
y = data[target]

# Apply preprocessing to the entire dataset
X = preprocessor.fit_transform(X)

def build_model(input_dim):
    model = Sequential([
        Input(shape=(input_dim,)),
        Dense(10, activation='relu'),
        Dense(3, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def calculate_metrics(y_true, y_pred):
    y_pred_rounded = y_pred.round()
    tp = np.sum((y_true == 1) & (y_pred_rounded == 1))
    tn = np.sum((y_true == 0) & (y_pred_rounded == 0))
    fp = np.sum((y_true == 0) & (y_pred_rounded == 1))
    fn = np.sum((y_true == 1) & (y_pred_rounded == 0))

    accuracy = (tp + tn) / (tp + tn + fp + fn)
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0

    return accuracy, precision, sensitivity, specificity

# 10-Fold Cross-Validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)
cross_val_metrics = []
fold = 1

for train_index, test_index in kf.split(X):
    print(f"\nFold {fold}:")
    fold += 1

    # Split data into train and test for this fold
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Build and train the model
    model = build_model(input_dim=X_train.shape[1])
    history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

    # Evaluate the model on the test set
    test_predictions = model.predict(X_test, verbose=0).flatten()
    accuracy, precision, sensitivity, specificity = calculate_metrics(y_test.values, test_predictions)

    cross_val_metrics.append((accuracy, precision, sensitivity, specificity))
    print(f"Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Sensitivity: {sensitivity:.4f}, Specificity: {specificity:.4f}")

# Print the average metrics across all folds
avg_metrics = np.mean(cross_val_metrics, axis=0)
print(f"\nAverage Metrics:\nAccuracy: {avg_metrics[0]:.4f}, Precision: {avg_metrics[1]:.4f}, Sensitivity: {avg_metrics[2]:.4f}, Specificity: {avg_metrics[3]:.4f}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

Fold 1:
Epoch 1/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 28ms/step - accuracy: 0.6933 - loss: 0.6978 - val_accuracy: 0.4925 - val_loss: 0.6916
Epoch 2/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7115 - loss: 0.6915 - val_accuracy: 0.4925 - val_loss: 0.6892
Epoch 3/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7356 - loss: 0.6858 - val_accuracy: 0.4925 - val_loss: 0.6870
Epoch 4/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8250 - loss: 0.6804 - val_accuracy: 0.5821 - val_loss: 0.6849
Epoch 5/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9378 - loss: 0.6756 - val_accuracy: 0.5821 - val_loss: 0.6831
Epoch 6/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9378 - loss: 0.6713 - val_accuracy: 0.5821 - val_loss: 0.6823
Epoch 7/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9378 - loss: 0.6675 - val_accuracy: 0.5821 - val_loss: 